In [68]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from IPython.display import display as Markdown
from tqdm.autonotebook import tqdm as notebook_tqdm

In [69]:
import os
import chardet
from langchain_community.document_loaders import (
    PyPDFLoader,
    Docx2txtLoader,
    CSVLoader,
    UnstructuredExcelLoader,
    TextLoader,
    UnstructuredMarkdownLoader
)

def process_folder(folder_path):
    documents = []
    supported_extensions = {
        '.pdf': PyPDFLoader,
        '.docx': Docx2txtLoader,
        '.csv': CSVLoader,
        '.xlsx': UnstructuredExcelLoader,
        '.txt': TextLoader,
        # '.md': TextLoader,
        # '.markdown': TextLoader
    }
    
    # 为代码文件单独处理
    code_extensions = {'.py', '.java', '.js', '.cpp', '.c', '.h', '.hpp', '.md', '.markdown'}

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            file_extension = os.path.splitext(file)[1].lower()
            
            try:
                if file_extension in supported_extensions:
                    # 对于支持的文件类型，先检测编码
                    if file_extension in {'.txt', '.csv'}:  # 只对文本类型的文件检测编码
                        with open(file_path, 'rb') as f:
                            raw_data = f.read()
                            detected = chardet.detect(raw_data)
                            encoding = detected['encoding']
                        
                        loader_class = supported_extensions[file_extension]
                        if file_extension == '.csv':
                            loader = loader_class(file_path, encoding=encoding)
                        else:
                            loader = loader_class(file_path, encoding=encoding)
                    else:
                        # 对于非文本类型的文件（如pdf, docx等）使用默认加载器
                        loader_class = supported_extensions[file_extension]
                        loader = loader_class(file_path)
                    docs = loader.load()
                    
                elif file_extension in code_extensions:
                    # 对代码文件特殊处理
                    with open(file_path, 'rb') as f:
                        raw_data = f.read()
                        detected = chardet.detect(raw_data)
                        encoding = detected['encoding']
                    loader = TextLoader(file_path, encoding=encoding)
                    docs = loader.load()
                else:
                    continue

                # 添加额外的元数据
                for doc in docs:
                    file_path = os.path.relpath(file_path, folder_path)
                    formatted_content = f"""
文件名: {file}
文件路径: {file_path}
文件类型: {file_extension}
---
{doc.page_content}
"""
                    doc.page_content = formatted_content
                    doc.metadata.update({
                        'filename': file,
                        'extension': file_extension,
                        'relative_path': file_path
                    })
                documents.extend(docs)
                print(f"成功处理文件: {file}")
            except Exception as e:
                print(f"处理文件 {file_path} 时出错: {str(e)}")
    
    return documents

# 使用修改后的函数
def transform_documents(folder_path):
    # 直接返回处理后的文档列表
    return process_folder(folder_path)

In [70]:
local_path = "D:/Desktop/ollama_pdf_rag-main/documents_for_analyse"
data = transform_documents(local_path)


成功处理文件: CartPole_introduction.md
成功处理文件: p_iteration.py
成功处理文件: QLearning.py
成功处理文件: Sarsa.py
成功处理文件: v_iteration.py


In [71]:
for i in data:
    print(i.page_content)



文件名: CartPole_introduction.md
文件路径: CartPole_introduction.md
文件类型: .md
---
# Cartpole 介绍文档

## 什么是 Cartpole

Cartpole 是一个经典的强化学习问题，也被称为倒立摆问题。它通常用作强化学习算法的基准测试环境。该问题的基本目标是让一个杆子保持垂直平衡，避免倒下。系统由一个带有固定轨道的小车和一根连接到小车的杆子组成。通过控制小车的水平运动，可以影响杆子的角度，并试图让杆子保持垂直位置。

![动图封面](https://pic3.zhimg.com/v2-44077b4784f621dcf4e3556a455f9f32_b.jpg)

## 问题描述

Cartpole 环境的基本组成包括：

- **小车**：一个可以在水平面上前后移动的物体。
- **杆子**：一个垂直于小车的杆子，杆子上端与小车相连接。
- **控制输入**：通过向小车施加力（可以是向左或向右），控制小车的水平位置和速度。

系统的目标是通过施加适当的力来防止杆子倒下。每当杆子的角度偏离垂直位置超过某个阈值，或者小车移出轨道，任务就被认为失败。为了保持杆子平衡，小车需要不断调整其位置。

### 环境的状态

在标准的 Cartpole 环境中，系统的状态由以下四个变量描述：

1. **小车的位置 (x)**：小车在轨道上的位置。
2. **小车的速度 (x')**：小车在水平方向上的速度。
3. **杆子的角度 (θ)**：杆子与垂直线的夹角。
4. **杆子的角速度 (θ')**：杆子旋转的速度。

这些状态变量可以通过与环境的交互得到，并用于指导强化学习算法进行决策。

### 动作空间

Cartpole 环境中有两个可用的动作：

- **向左施加力**（-1）：将小车向左推动。
- **向右施加力**（+1）：将小车向右推动。

每个动作的效果会影响小车的速度、位置以及杆子的角度，进一步决定是否能够成功保持平衡。

### 奖励函数

在 Cartpole 环境中，奖励函数是基于杆子是否保持平衡以及小车是否仍在轨道内来设计的。具体而言：

- 每一个时间步，小车保持杆子垂直位置会获得一个 +1 的奖励。
- 如果杆子的角度超过某个阈值（ ±12°），或者小车越过轨道边界

## Vector Embeddings

In [72]:
!ollama list

NAME                        ID              SIZE      MODIFIED    
deepseek-r1:latest          0a8c26691023    4.7 GB    3 weeks ago    
nomic-embed-text:latest     0a109f422b47    274 MB    4 weeks ago    
mxbai-embed-large:latest    468836162de7    669 MB    4 weeks ago    
qwen2.5-coder:latest        2b0496514337    4.7 GB    4 weeks ago    
qwen2.5:latest              845dbda0ea48    4.7 GB    4 weeks ago    


In [74]:
# # 1. First clean up any existing ChromaDB installations
# %pip uninstall -y chromadb
# %pip uninstall -y protobuf

# # 2. Install specific versions known to work together
# %pip install -q protobuf==3.20.3
# %pip install -q chromadb==0.4.22  # Using a stable older version
# %pip install -q langchain-ollama

# 3. Set the environment variable
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

In [75]:
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
import chromadb

In [76]:
# Split and chunk
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)
chromadb.api.client.SharedSystemClient.clear_system_cache()

In [2]:
# 5. Try creating the vector database
# vector_db = Chroma.from_documents(
#     documents=chunks,
#     embedding=OllamaEmbeddings(model="nomic-embed-text"),
#     collection_name="local-rag"
# )
from langchain_huggingface import HuggingFaceEmbeddings
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
embeddings_path = 'BAAI/bge-large-zh-v1.5'
# embeddings_path = "/home/ubuntu/embedding_models/bge-large-zh-v1.5"
embeddings = HuggingFaceEmbeddings(
    model_name=embeddings_path,
    model_kwargs={
        'device': device,
    },
    encode_kwargs={
        'normalize_embeddings': True,
        'batch_size': 32
    }
)

## Retrieval

In [78]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [79]:
# LLM from Ollama
# local_model = "llama3.2"
local_model = "deepseek-r1"
# local_model = "qwen2.5"
llm = ChatOllama(model=local_model)

In [80]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""你是一个 AI 语言模型助手。你的任务是为用户的问题生成五个不同的变体版本，
    用于从向量数据库中检索相关文档。通过生成多个不同角度的问题，你的目标是帮助用户克服
    基于距离相似度搜索的一些局限性。请提供这些替代性问题，每个问题用换行符分隔。
    原始问题: {question}""",
)
# retriever = MultiQueryRetriever.from_llm(
#     vector_db.as_retriever(), 
#     llm,
#     prompt=QUERY_PROMPT
# )
retriever = vector_db.as_retriever()

In [81]:
# RAG prompt

template = """回答问题基于以下文档:
{context}
问题: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [82]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
from IPython.display import Markdown, display

s = chain.invoke("请给出用QLearning算法解决CartPole问题的完整，可以直接运行的Python代码")
# s = chain.invoke("what is the happiest country in the world?")
display(Markdown(s))

<think>
嗯，我现在要解决的是使用QLearning算法来解决CartPole问题，并且给出一个可以直接运行的Python代码。我对CartPole问题有一定的了解，它是一个经典的控制任务，涉及到小车和杆子的状态，目标是让杆子保持垂直。

首先，我需要理解CartPole的基本结构和状态空间。根据问题描述，状态由四个变量组成：小车的位置、速度、杆子的角度以及杆子的角速度。这四个状态变量将被编码为一个四维向量，作为Q表格中的键。

接下来是动作空间。CartPole有两个动作，向左施加力（-1）和向右施加力（+1）。每个动作会改变环境的状态，影响未来的奖励。

QLearning的基本原理是通过经验更新来逐步逼近最优策略。我需要设计一个Q表格，其中每一行对应一个状态，每一列对应一个动作，存储的是该状态下采取该动作的预期总回报。初始化时，可以将所有Q值设为0，表示对各个状态-动作组合的初始期望都不清楚。

在选择动作的时候，我需要采用一种策略来决定是在探索还是利用。通常会使用ε-greedy策略，即以概率ε选择随机的动作，以概率1-ε选择当前最优的动作。这样可以平衡探索和利用之间的关系。

训练过程将包括多次 episode，每次 episode 从初始状态开始，按照当前策略选择动作，执行动作后转移到下一个状态，并根据奖励更新Q值。更新公式通常是 Q[state, action] = Q[state, action] + α * (reward + γ * max(Q[next_state]) - Q[state, action])，其中α是学习率，γ是折扣因子。

关于参数的选择， CartPole 的状态空间比较大，所以需要对状态进行离散化处理。这意味着将连续的状态值分割成有限的区间，并用整数来表示这些区间。例如，小车的位置可能被分为几个区间，每个区间对应一个离散的状态点。

在编码实现时，首先需要对各个状态变量进行归一化和离散化。这包括小车位置、速度、杆子角度和杆子角速度。由于这些值的范围不同，可能需要分别处理，并确定每个维度上的分割点。

接下来是初始化Q表格。考虑到状态被离散化后变成有限的状态数目，Q表可以是一个字典或二维列表。在Python中使用字典比较灵活，但为了高效访问，可以用数组结构或者字典的元组键。

训练循环中，逐步迭代，更新策略和Q值，直到达到预设的终止条件，如最大训练次数或达到稳定的奖励值。

最后是测试阶段，一旦模型训练完成后，需要在没有贪心策略的情况下运行，看看能否稳定地完成任务。或者可以使用最终 learned 的Q表来选择动作，执行整个 episode 并记录结果。

现在，我需要将这些步骤整合成完整的Python代码，并确保每一步都能正确实现。特别是离散化状态和训练过程中的更新逻辑容易出错，需要仔细检查。
</think>

以下是使用 Q-Learning 算法解决 CartPole 问题的完整 Python 代码：

```python
import gym
import numpy as np
from collections import defaultdict

def main():
    # 定义 CartPole 环境和参数
    env = gym.make('CartPole-v1')
    
    # 离散化状态空间，每个维度分割为若干区间
    state_dim = [20, 20, 20, 20]  # 小车位置、速度、杆子角度、角速度的分段数
    
    # 定义各状态变量的范围
    cart_pos_low = -4.8
    cart_pos_high = 4.8
    cart_vel_low = -3.14  # 原本应该是对称的，改为-2.4？或者其他值？
    cart_vel_high = 3.14
    
    pole_angle_low = -0.5
    pole_angle_high = 0.5  # 即±12度（因为 CartPole 的最大摆角是大约 ±12 度）
    pole_angvel_low = -8.72  # 原本的±4.0？
    pole_angvel_high = 8.72
    
    # 离散化函数
    def state_to_index(state):
        pos, vel, angle, angvel = state
        
        idx0 = np.digitize(pos, np.linspace(cart_pos_low, cart_pos_high, state_dim[0]))
        idx1 = np.digitize(vel, np.linspace(cart_vel_low, cart_vel_high, state_dim[1]))
        idx2 = np.digitize(angle, np.linspace(pole_angle_low, pole_angle_high, state_dim[2]))
        idx3 = np.digitize(angvel, np.linspace(-8.72, 8.72, state_dim[3]))
        
        return (idx0, idx1, idx2, idx3)
    
    # 初始化 Q 表格
    Q = defaultdict(lambda: [0.0] * 2)  # 每个状态有两个可能的动作
    
    # 超参数
    alpha = 0.1  # 学习率
    gamma = 0.99  # 折扣因子
    epsilon = 0.1  # 探索率
    num_episodes = 2000
    max_steps_per_episode = 1000
    
    best_score = float('-inf')
    
    for episode in range(num_episodes):
        state = env.reset()
        score = 0
        
        if (episode + 1) % 50 == 0:
            print(f"Episode {episode + 1}/{num_episodes}")
        
        while True:
            # 根据当前策略选择动作（epsilon-greedy）
            if np.random.random() < epsilon:
                action = np.random.choice([0, 1])
            else:
                state_key = tuple(state_to_index(state))
                current_q = Q[state_key]
                action = np.argmax(current_q + 1e-6)  # 加一个小的数避免全零
            
            next_state, reward, done, _ = env.step(action)
            score += reward
            
            # 离散化下一个状态
            next_state_key = tuple(state_to_index(next_state))
            
            if done:
                gamma = 0.9  # 终止状态的奖励处理
                
            # 更新 Q 表格
            current_q = Q[state_key]
            next_max_q = np.max(Q[next_state_key])
            
            old_q = current_q[action]
            new_q = old_q + alpha * (reward + gamma * next_max_q - old_q)
            Q[state_key][action] = new_q
            
            state = next_state
            
            if done or score >= 100:  # 如果完成任务或累积奖励达到100
                break
        
        if score > best_score:
            best_score = score
            print(f"Episode {episode + 1}: Best score {best_score}")
    
    # 最终测试
    test_state = env.reset()
    while True:
        state_key = tuple(state_to_index(test_state))
        action = np.argmax(Q[state_key])  # 取最优动作
        
        next_test_state, reward, done, _ = env.step(action)
        
        if done:
            break
        
        test_state = next_test_state
    
    print("完成任务！")
    
if __name__ == '__main__':
    main()
```

### 解释

1. **环境初始化**：
   - 使用 gym 库加载 CartPole 环境。
   - 定义 Cart 和杆子的状态空间，将每个状态变量（位置、速度、角度和角速度）分别离散化为若干区间。

2. **离散化函数**：
   - 将连续的状态值转换为离散的索引，以便在 Q 表格中查找。

3. **Q 表格初始化**：
   - 使用字典存储每个状态-动作组合的 Q 值。键是状态索引元组，值是一个包含两个元素的列表（对应两个可能的动作）。

4. **超参数设置**：
   - 学习率 (alpha)、折扣因子 (gamma) 和探索率 (epsilon) 是训练过程中的重要参数。

5. **训练循环**：
   - 迭代一定次数，每个 episode 从初始状态开始。
   - 使用 ε-greedy 策略选择动作，平衡探索和利用。
   - 执行动作后，根据奖励更新 Q 值。

6. **测试阶段**：
   - 训练完成后，在使用 learned 的 Q 表格选择最优动作的情况下，执行整个 CartPole 任务，验证模型是否成功完成。